In [1]:
import copy
import itertools
import os
import time

import matplotlib.pyplot as plt
import numpy as np
import pywt
from IPython.display import clear_output
from pynufft import NUFFT
from scipy import signal as sci_signal
from scipy.constants import c

from csromer.base import Dataset
from csromer.dictionaries.discrete import DiscreteWavelet
from csromer.dictionaries.undecimated import UndecimatedWavelet
from csromer.io import Reader, Writer
from csromer.objectivefunction import L1, TSV, TV, Chi2, OFunction
from csromer.optimization import BacktrackingFISTA, GradientBasedMethod
from csromer.reconstruction import Parameter
from csromer.simulation import FaradayThickSource, FaradayThinSource
from csromer.transformers import NDFT1D, NUFFT1D, Gridding
from csromer.utils import Gaussian, complex_to_real, real_to_complex

%matplotlib inline

#np.random.seed(666)

In [2]:
SMALL_SIZE = 17
MEDIUM_SIZE = 18
BIGGER_SIZE = 19

plt.rcParams.update(
    {
        "font.family": "serif",
        "text.usetex": True,
        "pgf.rcfonts": False,
        "pgf.texsystem": 'pdflatex',  # default is xetex
    }
)

plt.rc('font', size=MEDIUM_SIZE)  # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)  # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)  # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)  # legend fontsize

In [3]:
def gini_coefficient(w):
    # Order vector
    w_ordered = np.sort(np.abs(w), kind="stable")
    l1_norm = np.sum(np.abs(w_ordered))
    M = len(w_ordered)
    m = np.arange(0, M)
    const = (M - m + 1.5) / M
    if l1_norm == 0.0:
        coeff = np.nan
    else:
        coeff = np.sum((w_ordered / l1_norm) * const)
    return 1.0 - 2.0 * coeff

In [4]:
def chi2_calc(residuals):
    if residuals.dtype == np.complex64 or residuals.dtype == np.complex128:
        data = residuals.real**2 + residuals.imag**2
    else:
        data = residuals**2
    return np.sum(data)

In [5]:
def aicbic_normal(residuals, x):
    rss = chi2_calc(residuals)
    if x.dtype == np.complex64 or x.dtype == np.complex128:
        df = np.count_nonzero(x.real) + np.count_nonzero(x.imag)
    else:
        df = np.count_nonzero(x)

    l = 2 * len(residuals)
    aic = l * np.log(rss / l) + 2. * df
    bic = l * np.log(rss / l) + df * np.log(l)
    return aic, bic

In [6]:
#def aicbic(residuals, x):
#    rss = chi2_calc(residuals)
#    if x.dtype == np.complex64 or x.dtype == np.complex128:
#        df = np.count_nonzero(x.real) + np.count_nonzero(x.imag)
#    else:
#        df = np.count_nonzero(x)
#    df += 1  # Summing error term
#    l = 2 * len(residuals)

#    aic = l * np.log(rss / l) + 2. * df
#    if l - df != 1:
#        aic += 2. * df * (df + 1) / (l - df - 1)

#    bic = l * np.log(rss / l) + df * np.log(l)
#    return aic, bic


def aicbic(residuals, x):
    rss = chi2_calc(residuals)
    if x.dtype == np.complex64 or x.dtype == np.complex128:
        df = np.count_nonzero(x.real) + np.count_nonzero(x.imag)
    else:
        df = np.count_nonzero(x)
    l = 2 * len(residuals)
    return l * np.log(rss / l) + 2 * df, l * np.log(rss / l) + df * np.log(l)

In [7]:
def list_to2darray(x: list = None, cols: int = None, dtype=None):
    b = list(map(list, zip(*[iter(x)] * cols)))
    #b = [x[cols*i : cols*(i+1)] for i in range(rows)]
    if dtype is None:
        return np.array(b)
    else:
        return np.array(b, dtype=dtype)

In [8]:
class statistics:

    def __init__(self, m, n, z):
        self.sum = np.zeros((m, n), dtype=np.float32)
        self.sum2 = np.zeros((m, n), dtype=np.float32)
        self.n = z * np.ones((m, n), dtype=np.int32)

    def cumul(self, x):
        x_values = np.where(x != np.nan, x, 0.0)
        subtract = np.where(x == np.nan, -1, 0)
        self.sum += x_values
        self.sum2 += x_values * x_values
        self.n += subtract

    def mean(self):
        return np.where(self.n > 0, self.sum / self.n, np.nan)

    def std(self):
        return np.where(
            self.n > 0, np.sqrt(self.sum2 / self.n - self.sum * self.sum / self.n / self.n), np.nan
        )

In [10]:
class Test:

    def __init__(
        self,
        nu_min=None,
        nu_max=None,
        nchannels=None,
        noise_frac=None,
        remove_frac=None,
        use_gridding=False,
        ftransform="nufft",
        use_wavelet=None,
        source_1=None,
        source_2=None,
        scenario=1,
        append_signal=False,
    ):
        self.nu_min = nu_min
        self.nu_max = nu_max
        self.nchannels = nchannels
        self.noise_frac = noise_frac
        self.remove_frac = remove_frac
        self.use_gridding = use_gridding
        self.use_wavelet = use_wavelet
        self.ftransform = ftransform
        self.scenario = scenario
        self.append_signal = append_signal
        self.nu = np.linspace(start=nu_min, stop=nu_max, num=nchannels)
        self.source_1 = copy.deepcopy(source_1)
        self.source_2 = copy.deepcopy(source_2)

        if self.source_1 is not None:
            self.source_1.nu = self.nu
            self.source_1.simulate()

        if self.source_2 is not None:
            self.source_2.nu = self.nu
            self.source_2.simulate()

        if scenario == 1:
            self.source = self.source_1
        elif scenario == 2:
            self.source = self.source_2
        elif scenario == 3:
            self.source = self.source_1 + self.source_2
        else:
            raise ValueError("This scenario does not exist")

        if remove_frac:
            t = int(time.time() * 1000.0)
            seed = ((t & 0xff000000) >> 24) + ((t & 0x00ff0000) >> 8) + ((t & 0x0000ff00) << 8) + (
                (t & 0x000000ff) << 24
            )
            print("Seed removal {0}".format(seed))
            self.source.remove_channels(remove_frac, np.random.RandomState(seed))

        if scenario == 1:
            self.avg_signal = np.abs(self.source_1.s_nu)
        elif scenario == 2:
            self.avg_signal = np.abs(self.source_2.s_nu)
        else:
            self.avg_signal = (np.abs(self.source_1.s_nu) + np.abs(self.source_2.s_nu)) / 2.0

        if noise_frac:
            self.source.apply_noise(self.avg_signal * self.noise_frac)

        if use_gridding:
            gridding = Gridding(self.source)
            self.source = gridding.run()

    def apply_noise(self):
        if self.noise_frac:
            t = int(time.time() * 1000.0)
            seed = ((t & 0xff000000) >> 24) + ((t & 0x00ff0000) >> 8) + ((t & 0x0000ff00) << 8) + (
                (t & 0x000000ff) << 24
            )
            print("Seed noise {0}".format(seed))
            self.source.apply_noise(self.avg_signal * self.noise_frac, np.random.RandomState(seed))
            print("Calculating l2_0...")
            self.source.l2_ref = self.source.calculate_l2ref()
            print("l2_0: {0:0.3f} rad/m^2".format(self.source.l2_ref))

    def run(self, lambda_tv: float = None, lambda_tsv: float = None):
        self.parameter = Parameter()
        self.parameter.calculate_cellsize(dataset=self.source)

        dft = NDFT1D(dataset=self.source, parameter=self.parameter)

        self.F_dirty = dft.backward(self.source.data)

        if self.use_wavelet:
            #self.wavelet = DiscreteWavelet(
            #    wavelet_name=self.use_wavelet,
            #    mode="periodization",
            #    append_signal=self.append_signal
            #)
            self.wavelet = UndecimatedWavelet(
                wavelet_name=self.use_wavelet,
                mode="periodization",
                append_signal=self.append_signal
            )
            self.lambda_l1 = np.sqrt(len(self.source.data) + 2. * np.sqrt(len(self.source.data))
                                     ) * 2.0 * np.sqrt(2) * np.mean(self.source.sigma)
        else:
            self.lambda_l1 = np.sqrt(len(self.source.data) + 2. * np.sqrt(len(self.source.data))
                                     ) * np.sqrt(2) * np.mean(self.source.sigma)

        if lambda_tv is None:
            lambda_tv = 0.0

        if lambda_tsv is None:
            lambda_tsv = 0.0

        if self.ftransform == "nufft":
            nufft = NUFFT1D(dataset=self.source, parameter=self.parameter, solve=True)
            if self.use_wavelet:
                chi2 = Chi2(dft_obj=nufft, wavelet=self.wavelet)
            else:
                chi2 = Chi2(dft_obj=nufft)
        else:
            if self.use_wavelet:
                chi2 = Chi2(dft_obj=dft, wavelet=self.wavelet)
            else:
                chi2 = Chi2(dft_obj=dft)

        l1 = L1(reg=self.lambda_l1)
        tsv = TSV(reg=lambda_tsv)
        tv = TV(reg=lambda_tv)
        F_func = [chi2, l1]
        f_func = [chi2]
        g_func = [l1]

        F_obj = OFunction(F_func)
        f_obj = OFunction(f_func)
        g_obj = OFunction(g_func)

        self.parameter.data = self.F_dirty

        self.parameter.complex_data_to_real()

        if self.use_wavelet:
            self.parameter.data = self.wavelet.decompose(self.parameter.data)

        opt = BacktrackingFISTA(
            guess_param=self.parameter,
            F_obj=F_obj,
            fx=chi2,
            gx=g_obj,
            noise=2. * self.source.theo_noise,
            verbose=False
        )
        self.obj, self.X = opt.run()

        if self.use_wavelet is not None:
            self.coeffs = copy.deepcopy(self.X.data)
            self.k = np.count_nonzero(self.coeffs)
            self.sparsity = self.k / len(self.coeffs)
            self.gini = gini_coefficient(self.coeffs)
            self.X.data = self.wavelet.reconstruct(self.X.data)
            self.aic, self.bic = aicbic(self.source.residual, self.coeffs)

        else:
            self.k = np.count_nonzero(self.X.data)
            self.sparsity = self.k / len(self.X.data)
            self.gini = gini_coefficient(self.X.data)
            self.aic, self.bic = aicbic(self.source.residual, self.X.data)

        self.X.real_data_to_complex()

        self.X_residual = dft.backward(self.source.residual)

        self.X_restored = self.X.convolve() + self.X_residual

        self.res_noise = 0.5 * (np.nanstd(self.X_residual.real) + np.nanstd(self.X_residual.imag))
        self.rmse = np.sqrt(
            np.sum(self.source.residual.real**2 + self.source.residual.imag**2) /
            (2 * len(self.source.residual))
        )
        meaningful_signal = np.where(np.abs(self.parameter.phi) < self.parameter.max_faraday_depth)
        self.signal = np.mean(np.abs(self.X_restored[meaningful_signal]))
        self.peak_signal = np.nanmax(np.abs(self.X_restored))

        self.snr = self.signal / self.res_noise
        self.psnr = self.peak_signal / self.res_noise
        
        print("Data points: {0}".format(2*len(self.source.residual)))
        print("Non-zero coeffs {0}".format(self.k))
        print("Signal-to-noise ratio: {0}".format(self.snr))
        print("Peak Signal-to-noise ratio: {0}".format(self.psnr))
        print("Standard deviation: ({0})*10**-5".format(self.res_noise * 10**5))
        print("Root Mean Squared Error: ({0})*10**-5".format(self.rmse * 10**5))
        print("Sparsity {0}".format(self.sparsity))
        print("\n")
        """
        
        self.fig, self.ax = plt.subplots(nrows=2, ncols=4, sharey='row', figsize=(18, 5))

        # Data
        self.ax[0,0].plot(self.source.lambda2, self.source.data.real, 'k.', label=r"Stokes $Q$")
        self.ax[0,0].plot(self.source.lambda2, self.source.data.imag, 'c.', label=r"Stokes $U$")
        self.ax[0,0].plot(self.source.lambda2, np.abs(self.source.data), 'g.', label=r"$|P|$")
        self.ax[0,0].set_xlabel(r'$\lambda^2$[m$^{2}$]')
        self.ax[0,0].set_ylabel(r'Jy/beam')
        self.ax[0,0].title.set_text("Data")

        self.ax[1,0].plot(self.parameter.phi, self.F_dirty.real, 'c--', label=r"Stokes $Q$")
        self.ax[1,0].plot(self.parameter.phi, self.F_dirty.imag, 'c:', label=r"Stokes $U$")
        self.ax[1,0].plot(self.parameter.phi, np.abs(self.F_dirty), 'k-', label=r"|P|")
        self.ax[1,0].set_xlabel(r'$\phi$[rad m$^{-2}$]')
        self.ax[1,0].set_ylabel(r'Jy/beam m$^2$ rad$^{-1}$ rmtf$^{-1}$')
        self.ax[1,0].set_xlim([-1000,1000])

        # Model
        self.ax[0,1].plot(self.source.lambda2, self.source.model_data.real, 'k.', label=r"Stokes $Q$")
        self.ax[0,1].plot(self.source.lambda2, self.source.model_data.imag, 'c.', label=r"Stokes $U$")
        self.ax[0,1].plot(self.source.lambda2, np.abs(self.source.model_data), 'g.', label=r"$|P|$")
        self.ax[0,1].set_xlabel(r'$\lambda^2$[m$^{2}$]')
        self.ax[0,1].set_ylabel(r'Jy/beam')
        self.ax[0,1].title.set_text("Model")

        self.ax[1,1].get_shared_y_axes().remove(self.ax[1,1])
        self.ax[1,1].clear()
        self.ax[1,1].plot(self.parameter.phi, self.X.data.real, 'c--', label=r"Stokes $Q$")
        self.ax[1,1].plot(self.parameter.phi, self.X.data.imag, 'c:', label=r"Stokes $U$")
        self.ax[1,1].plot(self.parameter.phi, np.abs(self.X.data), 'k-', label=r"$|P|$")
        self.ax[1,1].set_xlabel(r'$\phi$[rad m$^{-2}$]')
        self.ax[1,1].set_ylabel(r'Jy/beam m$^2$ rad$^{-1}$ pix$^{-1}$')
        self.ax[1,1].set_xlim([-1000,1000])
        
        # Residual

        self.ax[0,2].plot(self.source.lambda2, self.source.residual.real, 'k.', label=r"Stokes $Q$")
        self.ax[0,2].plot(self.source.lambda2, self.source.residual.imag, 'c.', label=r"Stokes $U$")
        self.ax[0,2].plot(self.source.lambda2, np.abs(self.source.residual), 'g.', label=r"$|P|$")
        self.ax[0,2].set_xlabel(r'$\lambda^2$[m$^{2}$]')
        self.ax[0,2].set_ylabel(r'Jy/beam')
        self.ax[0,2].title.set_text("Residual")

        self.ax[1,2].plot(self.parameter.phi, self.X_residual.real, 'c--', label=r"Stokes $Q$")
        self.ax[1,2].plot(self.parameter.phi, self.X_residual.imag, 'c:', label=r"Stokes $U$")
        self.ax[1,2].plot(self.parameter.phi, np.abs(self.X_residual), 'k-', label=r"$|P|$")
        self.ax[1,2].set_xlabel(r'$\phi$[rad m$^{-2}$]')
        self.ax[1,2].set_ylabel(r'Jy/beam m$^2$ rad$^{-1}$ rmtf$^{-1}$')
        self.ax[1,2].set_xlim([-1000,1000])
        
        if self.use_wavelet:
            self.ax[0,3].get_shared_y_axes().remove(self.ax[0,3])
            self.ax[0,3].clear()
            self.ax[0,3].plot(self.coeffs)
            self.ax[0,3].title.set_text("Coefficients")

        self.ax[1,3].plot(self.parameter.phi, self.X_restored.real, 'c--', label=r"Stokes $Q$")
        self.ax[1,3].plot(self.parameter.phi, self.X_restored.imag, 'c:', label=r"Stokes $U$")
        self.ax[1,3].plot(self.parameter.phi, np.abs(self.X_restored), 'k-', label=r"$|P|$")
        self.ax[1,3].set_xlim([-1000,1000])
        self.ax[1,3].set_xlabel(r'$\phi$[rad m$^{-2}$]')
        self.ax[1,3].set_ylabel(r'Jy/beam m$^2$ rad$^{-1}$ rmtf$^{-1}$')
        self.ax[1,3].title.set_text("Restored")
        
        self.fig.tight_layout()
        """

In [11]:
def run_test(
    source_1,
    source_2,
    nsigma,
    remove_frac,
    nu_min=1.008e9,
    nu_max=2.031e9,
    nchannels=1000,
    scenario=1,
    use_wavelet=None,
    append_signal=False
):
    nosigma_objs = []
    for remv_frac in remove_frac:
        nosigma_objs.append(
            Test(
                nu_min=nu_min,
                nu_max=nu_max,
                nchannels=nchannels,
                noise_frac=0.0,
                remove_frac=remv_frac,
                scenario=scenario,
                source_1=source_1,
                source_2=source_2,
                use_wavelet=use_wavelet,
                append_signal=append_signal
            )
        )

    test_objs = []
    for nsig in nsigma:
        for i in range(0, len(remove_frac)):
            copy_object = copy.deepcopy(nosigma_objs[i])
            copy_object.noise_frac = nsig
            copy_object.apply_noise()
            test_objs.append(copy_object)

    del nosigma_objs

    nid = len(nsigma) * len(remove_frac)
    for _id in range(0, nid):
        test_objs[_id].run()

    return test_objs

In [12]:
# JVLA 1.008 - 2.031 GHz 546 channels
# MeerKAT 0.9 GHz-1.420 GHz 546
# eMERLIN 1.230 - 1.740 GHz 4096
def run_tests(
    source_1,
    source_2,
    nsigma,
    remove_frac,
    nsamples,
    nu_min=1.008e9,
    nu_max=2.031e9,
    nchannels=1000,
    scenario=1,
    use_wavelet=None,
    append_signal=False
):
    m = len(nsigma)
    n = len(remove_frac)
    psnrs = statistics(m, n, nsamples)
    rmses = statistics(m, n, nsamples)
    #noises = statistics(m, n, nsamples)
    sparsities = statistics(m, n, nsamples)
    #ginies = statistics(m, n, nsamples)
    aics = statistics(m, n, nsamples)
    bics = statistics(m, n, nsamples)

    for i in range(0, nsamples):
        test = run_test(
            source_1,
            source_2,
            nsigma,
            remove_frac,
            nu_min=nu_min,
            nu_max=nu_max,
            nchannels=nchannels,
            scenario=scenario,
            use_wavelet=use_wavelet,
            append_signal=append_signal
        )
        psnrs.cumul(list_to2darray([x.psnr for x in test], n, dtype=np.float32))
        #noises.cumul(list_to2darray([x.res_noise for x in test], n, dtype=np.float32))
        sparsities.cumul(list_to2darray([x.sparsity * 100.0 for x in test], n, dtype=np.float32))
        #ginies.cumul(list_to2darray([x.gini for x in test], n, dtype=np.float32))
        rmses.cumul(list_to2darray([x.rmse for x in test], n, dtype=np.float32))
        aics.cumul(list_to2darray([x.aic for x in test], n, dtype=np.float32))
        bics.cumul(list_to2darray([x.bic for x in test], n, dtype=np.float32))
        for t in test:
            del t
        test = []

    psnr_mean, psnr_std = psnrs.mean(), psnrs.std()
    rmse_mean, rmse_std = rmses.mean(), rmses.std()
    #noise_mean, noise_std = noises.mean(), noises.std()
    sparsity_mean, sparsity_std = sparsities.mean(), sparsities.std()
    #gini_mean, gini_std = ginies.mean(), ginies.std()
    aic_mean, aic_std = aics.mean(), aics.std()
    bic_mean, bic_std = bics.mean(), bics.std()
    return psnr_mean, psnr_std, rmse_mean, rmse_std, aic_mean, aic_std, bic_mean, bic_std, sparsity_mean, sparsity_std

In [13]:
source_1 = FaradayThinSource(s_nu=0.0035, phi_gal=-200, spectral_idx=1.0)
source_2 = FaradayThickSource(s_nu=0.0035, phi_fg=140, phi_center=200, spectral_idx=1.0)

In [14]:
nsigma = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
remove_frac = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
scenario = 3
use_wavelet = "haar"
#use_wavelet = None
append_signal = False
samples = 50

In [15]:
psnr_mean, psnr_std, rmse_mean, rmse_std, aic_mean, aic_std, bic_mean, bic_std, sparsity_mean, sparsity_std = run_tests(
    source_1,
    source_2,
    nsigma,
    remove_frac,
    samples,
    scenario=scenario,
    use_wavelet=use_wavelet,
    append_signal=append_signal
)

Seed removal 1173710626
Seed removal 1307928354
Seed removal 1442146082
Seed removal 1509254946
Seed removal 1609918242
Seed removal 1677027106
Seed removal 1727358754
Seed removal 1777690402
Seed removal 1811244834
Seed noise 1828022050
Calculating l2_0...
l2_0: 0.043 rad/m^2
Seed noise 1828022050
Calculating l2_0...
l2_0: 0.043 rad/m^2
Seed noise 1844799266
Calculating l2_0...
l2_0: 0.046 rad/m^2
Seed noise 1861576482
Calculating l2_0...
l2_0: 0.041 rad/m^2
Seed noise 1878353698
Calculating l2_0...
l2_0: 0.041 rad/m^2
Seed noise 1895130914
Calculating l2_0...
l2_0: 0.040 rad/m^2
Seed noise 1895130914
Calculating l2_0...
l2_0: 0.040 rad/m^2
Seed noise 1911908130
Calculating l2_0...
l2_0: 0.052 rad/m^2
Seed noise 1928685346
Calculating l2_0...
l2_0: 0.049 rad/m^2
Seed noise 1945462562
Calculating l2_0...
l2_0: 0.043 rad/m^2
Seed noise 1945462562
Calculating l2_0...
l2_0: 0.043 rad/m^2
Seed noise 1962239778
Calculating l2_0...
l2_0: 0.046 rad/m^2
Seed noise 1979016994
Calculating l2_0..


KeyboardInterrupt



In [ ]:
means = [psnr_mean, rmse_mean, aic_mean, bic_mean]
stds = [psnr_std, rmse_std, aic_std, bic_std]
names = ["PSNR", "RMSE", "AIC", "BIC"]

In [ ]:
import os

if use_wavelet is None:
    ofile_name = "deltaf_basis_scenario_" + str(scenario) + ".pdf"
else:
    ofile_name = "undecimated_wavelet_scenario_" + str(scenario) + ".pdf"
fig, ax = plt.subplots(nrows=1, ncols=4, sharey='none', sharex='all', figsize=(18, 5))
cmap = plt.get_cmap('tab20')

for f in range(4):
    number = means[f].shape[1]
    colors = [cmap(i) for i in np.linspace(0, 1, number)]

    for i, color in enumerate(colors, start=0):
        lowers = np.minimum(stds[f][:, i], means[f][:, i])
        ax[f].plot(nsigma, means[f][:, i], label=str(remove_frac[i]), color=color)
        if f < 2:
            ax[f].errorbar(
                nsigma,
                means[f][:, i],
                yerr=[lowers, stds[f][:, i]],
                fmt='.',
                capsize=4,
                color=color
            )
        else:
            ax[f].errorbar(
                nsigma, means[f][:, i], yerr=stds[f][:, i], fmt='.', capsize=4, color=color
            )
        #ax[f].fill_between(nsigma, means[f][:,i]+stds[f][:,i], color=color, alpha=.5)
        #ax[f].fill_between(nsigma, means[f][:,i]-stds[f][:,i], color=color, alpha=.5)

    #if scenario == 1 and names[f] == "BIC":
    #    ax[f].legend(bbox_to_anchor=(1., 1.02), ncol=2, fancybox=True, shadow=True, title="Removal fraction")

    if scenario == 3:
        ax[f].set_xlabel("Noise fraction")
    else:
        ax[f].xaxis.set_ticklabels([])

    ax[f].set_ylabel(names[f])

ax[2].set_ylim([-40000, 0])
h, l = ax[0].get_legend_handles_labels()
if scenario == 3:
    legend = fig.legend(
        h,
        l,
        loc='lower center',
        ncol=10,
        fancybox=True,
        bbox_to_anchor=[0.5, -0.2],
        shadow=True,
        title="Removal fraction"
    )
fig.tight_layout()
if os.path.isfile(ofile_name):
    os.remove(ofile_name)  # Opt.: os.system("rm "+strFile)
fig.savefig(ofile_name, dpi=100, bbox_inches='tight')

In [ ]:
# [0.2, 0.5, 0.8] Removal fraction [2, 5, 8]
# [0.2, 0.5, 0.8] Noise # [2, 5, 8]
"""
for i in [2, 5, 8]:
    for j in [2, 5, 8]:
        idx = len(remove_frac)*i + j
        #print(idx)
        selected_test = test_objs[idx]
        print(np.max(selected_test.source.lambda2))
        plt.clf()
        plt.plot(selected_test.source.lambda2, selected_test.source.data.real, 'k.', label=r"Stokes $Q$")
        plt.plot(selected_test.source.lambda2, selected_test.source.data.imag, 'c.', label=r"Stokes $U$")
        plt.plot(selected_test.source.lambda2, np.abs(selected_test.source.data), 'g.', label=r"$|P|$")
        if i==8:
            plt.xlabel(r'$\lambda^2$[m$^{2}$]')
        else:
            plt.xticks([])
        if j==2:
            plt.ylabel(r'Jy/beam')
        else:
            plt.yticks([])
        #plt.legend(loc=(1.04, 0.755), fancybox=True, shadow=True)
        plt.xlim([0.0,0.09])
        plt.ylim([-0.01,0.01])
        plt.tight_layout()
        plt.savefig("pol_sim_rem_"+str(remove_frac[j])+"_noise_"+str(nsigma[i])+"_"+str(idx)+".png", dpi=100)
"""